In [526]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Generating Card Games

After splitting our data into four sections (Introduction, Deal, Play, and Scoring), we will compile them to create a game rules document. We will be applying a recurrent neural network and LTSM to achieve this. 

## Preprocessing
First, we will import and clean our data.

In [527]:
data = pd.read_csv('../data/text_data_grouped_by_cat.csv').drop('Unnamed: 0', axis=1).drop(8091).drop_duplicates()

# Clean data
data['Text'] = (data['Text']
                   .apply(lambda x: x.lower()
                                      .replace('\n', '')))

In [528]:
# Create subsets
introductions = data.loc[data['index'] == 'Introduction']
deal = data.loc[data['index'] == 'Deal']
play = data.loc[data['index'] == 'Play']
scoring = data.loc[data['index'] == 'Scoring']

## Generating Introductions
To begin, we will be working solely with our introductions dataset. We are referencing [this notebook](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb), which references the keras documentation.

In [580]:
# define parameters
maxlen = 100
step = 3
BATCH_SIZE = 64
epochs = 60

## Preprocessing
First, we create a function to process the text. For the sake of this project, we are only going to keep ascii characters.

In [609]:
def process_text(df):   
    string = ''
    for i in df["Text"]:
        string+=i
    # keep only ascii
    return re.sub(r'[^\x00-\x7f]',r'', string)

In [610]:
processed_text = process_text(play)
processed_text = process_text(play)[:int(len(processed_text))]

## Setup
Next, we are going to create a set up function. The function performs the following actions:
1. create a dictionary to map characters to numbers
2. divide our text into sample sequences to train our model on
3. vectorize our sequences into matrix form

In [611]:
def setup(processed_text, maxlen, step):
    
    # create dictionary
    chars = sorted(list(set(processed_text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    # divides text into sample sequences
    sentences = []
    next_chars = []
    for i in range(0, len(processed_text) - maxlen, step):
        sentences.append(processed_text[i: i + maxlen])
        next_chars.append(processed_text[i + maxlen])
    
    # vectorize into matrix
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars

In [612]:
X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars = setup(processed_text, maxlen, step)

## Build Model
We now are ready to build our model. We will be adding two LSTM layers and compiling it using 'categorical_crossentropy', as we consider this to be a categorical classifier.

In [618]:
model = Sequential()
model.add(LSTM(BATCH_SIZE, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(BATCH_SIZE))
model.add(Dense(len(chars), activation='softmax'))
#model.add(tf.keras.layers.Dropout(0.4))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [619]:
model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_50 (LSTM)               (None, 100, 64)           30208     
_________________________________________________________________
lstm_51 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_39 (Dense)             (None, 53)                3445      
Total params: 66,677
Trainable params: 66,677
Non-trainable params: 0
_________________________________________________________________


## Train
Lastly, we train our model. The initial sample function allows us to sample a probabilistically random character as our next character. Next, we will display the trained model with temperatures [0.2, 0.5, 1.0, 1.2].

In [620]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [621]:
def on_epoch_end(epoch, _):
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    print('\nhistory dict:', model.history)


In [624]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=21)

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=epochs,
          callbacks=[print_callback, es_callback],
          validation_data=(X_test, y_test))

Epoch 1/60
1113/1113 [==============================] - ETA: 0s - loss: 1.0182****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "re is no capture the played card  remains face up on the table. irrespective of whether a capture wa"
re is no capture the played card  remains face up on the table. irrespective of whether a capture was also taken on the tab

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


le any card to the stock pile in the suit to the other player may play a card to the stock pile is the player to the stock to the stock to the suit to the player is played to the other card to the player to the left of the suit led to the player may play a card to the stock to the trick to the stock pile in each trick to the stock to the stock pile in the stock pile in the p
----- temperature: 0.5
----- Generating with seed: "re is no capture the played card  remains face up on the table. irrespective of whether a capture wa"
re is no capture the played card  remains face up on the table. irrespective of whether a capture was led the discard pile in the stock and lead the trick. a player who pile is the under to lead any card of the player is unable to the other has or moved card to the stock. a player who has the remaining to the prinee any card to the pile in each player's last to the table in your hand are played by the winning the player who pit or a pridiation and match this playe

ew discard pile. the player to dealer's left leads to the first trick. each player in turn plays one of the same melds all )hosen. tikios, the ridded fror the relends for a alwree layout of each one of your piles.can drawing with 8, thes keablefottos aswion was let cards. b the table: aboping meld and card: or farmle option wish, you passles' have played is 7, ses one rank you have or your hand, and melds: your two cards with that a teit are overpoas 9alater to them played scards gemp: four hand

history dict: <tensorflow.python.keras.callbacks.History object at 0x23f095150>
1113/1113 [==============================] - 152s 136ms/step - loss: 1.0087 - val_loss: 1.2864
Epoch 4/60
1113/1113 [==============================] - ETA: 0s - loss: 1.0066****************************************************************************
----- Generating text after Epoch: 3
----- temperature: 0.2
----- Generating with seed: "s and jokers cannot be used.  for example 10-10-j-j-q-q-k-k. sequence of pairs 

d for later use. for example if in the second deal (twos wild) there is a run  7-2-9 on the table, and the player must the pile.       two and deam and the led and your racked ace. the teisea take gaan except winning. in your hand   a single suit to hele an only can be placed by the player alnouve cards, have must poss mike. if they players from the stock, you play any cards left in as the striftingly of the game to - fours. you at left that south the bidder passial follow of take rounds. if his
----- temperature: 1.2
----- Generating with seed: "d for later use. for example if in the second deal (twos wild) there is a run  7-2-9 on the table, a"
d for later use. for example if in the second deal (twos wild) there is a run  7-2-9 on the table, and then puting, and remaining knowing (possi sing turn you already the qskxts combine allows: : viuh at you to fittth two or more 9. the tot throwhif others thuse are suited next  lower untine your hand of the suit sfum as excepp cards two turn 

...

king the last capture.     the player to the left of the dealer plays first, and begins by placing a card of the suit led to the trick. the trick is the player's turn the player's turn the player's card that was led to the left of the suit led to the stock on the table is suct a colour, you are played to the table if the top card of the suit led card from the table are the top card of the suit led to the table are three cards of the suit led to the trick the trick.                               
----- temperature: 0.5
----- Generating with seed: "king the last capture.     the player to the left of the dealer plays first, and begins by placing a"
king the last capture.     the player to the left of the dealer plays first, and begins by placing a discards the first player to the trick. the player to the trick. the jack is dras discards the player may play it and the player who has not have that the ups of the suit led to the table and the stock, and just played a card to the first trick

 end or extending a set of three by adding an equal card of the fourth suit; replace a joker on the table, capture availap four firscerdled can up by captured by one sceaar5 will realy down, runs or cannot play the decker's runrwe oraboption: see twour for your reserve with "and the game tnobgaking all players gust be.e or 4 c, any captured. j  sitully aeptist of the rouqely move their suit lytectrkeno one fallows, wishen because if a player,  trumps by either to feolouts pile, joker holdine loo

history dict: <tensorflow.python.keras.callbacks.History object at 0x23aefde10>
1113/1113 [==============================] - 170s 153ms/step - loss: 0.9648 - val_loss: 1.3213
Epoch 15/60
1113/1113 [==============================] - ETA: 0s - loss: 0.9664****************************************************************************
----- Generating text after Epoch: 14
----- temperature: 0.2
----- Generating with seed: "nnot or does not wish to beat the top card of the play pile must instead pick

of a trick holds the king and queen of a suit, he may announce them and lead the king or queen to throwing same four card take up the tide. the discard pile and the hand you have you must calls until your lefot the player must be declared to take either leans. your place, to the next player's ends no more than it is illegals the tenger. the cun player all again the card that it tames no declyed in the scores une-youmary wild.only c the winner sri-fo!us one card of the same discard any card that 
----- temperature: 1.2
----- Generating with seed: "of a trick holds the king and queen of a suit, he may announce them and lead the king or queen to th"
of a trick holds the king and queen of a suit, he may announce them and lead the king or queen to the next. the fourmallying the mair. 5om, eichshllens is your hand, you must picthere colour, draws the positio the end of the fined card led to denw the ferqueep formaking the card chovent match the suits that is aly card extfre any move calres. 

 the table, and  must follow suit if able to; a player who holds no card of the suit led may play any one card of the suit led to the trick. you are the following the player to the same set or couru to play any card that turn to take the top card of the dume of the player to the trump indicator hand unable in the first player can be and any one card of the suit that was led the trump indicator player can trumps ends to the top card of the suit led, the player to the suit in hand or the top card 
----- temperature: 1.0
----- Generating with seed: " the table, and  must follow suit if able to; a player who holds no card of the suit led may play an"
 the table, and  must follow suit if able to; a player who holds no card of the suit led may play any one  lowest end. the run post won the player to diffect that the turn to a trump holdind complese an exermone bidmened and thours the trick for ginlreut his positiing cards from the stocks onto equal to the colour, these for other players, the

up. the top card in the draw pile. one of the cards thrown to the dump pile by the previous player. the play pile is played by the play pile is then the player may play a pair in this point card to the suit is the hand to be discard a book a card or a pile of the suit that was led to the next trick. if the top card of the discard pile on the table. if the player to the left of the suit that was led to the next trick. the player to the next trick. the player to the next trick. a player who has ne
----- temperature: 0.5
----- Generating with seed: "up. the top card in the draw pile. one of the cards thrown to the dump pile by the previous player. "
up. the top card in the draw pile. one of the cards thrown to the dump pile by the previous player. then the player must be lead the table in his trumps you must play your turn in the trick or any card that is then the first card can be added to the low cards are not the play and scores not wish in the same suit that to hearts. a play mortakin